# Performing Read Alignment

Here we will use the BWA aligner to align a smll set of Illumina sequencing data to the _Mus Musculus_ reference genome. We will align genomic sequence (from Whole-Genome Sequencing) from a mouse embryo which has been mutagenised while the one-cell stage using CRISPR-Cas9 and a gRNA targeting an exon of the Tyr gene. The successful mutation of the gene will delete one or both alleles. A bi-allelic null Tyr mouse will be albino, but otherwise healthy.

First, check you are in the correct directory.

In [ ]:
pwd

It should display something like:

  ``/home/manager/course_data/read_alignment``

## Viewing the reference genome

Go to the `ref` directory that contains the fasta files of the reference genomes:

In [ ]:
cd ~/course_data/read_alignment/data/ref

Fasta files (.fa) are used to store raw sequencing information before aligning data. A single chromosome from the mouse genome is contained in the file GRCm38.68.dna.toplevel.chr7.fa.gz

View the file with zless (we use zless instead of less because the file is compressed):

In [ ]:
zless GRCm38.68.dna.toplevel.chr7.fa.gz

__Q1: What is the length of chromosome 7 of the mouse genome? (Hint: Look at the fasta header for chromosome 7)__

Similar to a BAM file, to allow fast retrieval of data, an index file is often required. You should check for the presencen of fasta indexes for the genome in the 'ref' directory:

``GRCm38.68.dna.toplevel.chr7.fa.gz.amb ... GRCm38.68.dna.toplevel.chr7.fa.gz.sa``

These are created by BWA: suffixtrees, bwt transform etc etc.
    
If these index files don’t exist, then you can run the indexing with the command

``bwa index GRCm38.68.dna.toplevel.chr7.fa.gz`` 
    
Beware – this indexing process can take 3-5 minutes so please only run it if the index files do not exist!

## Align the data with bwa

Go to the `~/course_data/read_alignment/data/Exercise1/fastq/` directory - you can use this command: 

In [ ]:
cd ../Exercise1/fastq

Use the `bwa mem` command to align the fastq files to the mouse reference genome. By default bwa outputs SAM format directly to the standard output (in this case your terminal window), therefore you will have to redirect the result into a SAM file.

In [ ]:
bwa mem ../../ref/GRCm38.68.dna.toplevel.chr7.fa.gz md5638a_7_87000000_R1.fastq.gz md5638a_7_87000000_R2.fastq.gz > md5638.sam

This may take a few minutes, please be patient.

## Convert a SAM file to a BAM file

Now use samtools to convert the SAM file `md5638.sam` created in the previous step into a BAM file called `md5638.bam`.

In [ ]:
samtools view -O BAM -o md5638.bam md5638.sam

__Q2: How much space is saved by using a BAM file instead of a SAM file?__

## Sort and index the BAM file

The BAM files produced by BWA are sorted by read name (same order as the original fastq files). However, most viewing and variant calling software require the BAM files to be sorted by reference coordinate position and indexed for rapid retrieval. Therefore, use ‘samtools sort’ to produce a new BAM file called `md5638.sorted.bam` that is sorted by position.

In [ ]:
samtools sort -T temp -O bam -o md5638.sorted.bam md5638.bam

Finally index the sorted BAM file using ‘samtools index’ command. 

__Note:__ indexing a BAM file is also a good way to check that the BAM file has not been truncated (e.g. your disk becomes full when writing the BAM file). At the end of every BAM file, a special end of file (EOF) marker is written. The Samtools index command will first check for this and produce an error message if it is not found.

In [ ]:
samtools index md5638.sorted.bam

## Unix pipes to combine the commands together

To produce the sorted BAM file above we had to carry out several separate commands and produce intermediate files. The Unix pipe command allows you to feed the output of one command into the next command.

You can combine all of these commands together using unix pipes, and do all of this data processing together and avoid writing intermediate files. To do this type:

In [ ]:
bwa mem ../../ref/GRCm38.68.dna.toplevel.chr7.fa.gz md5638a_7_87000000_R1.fastq.gz md5638a_7_87000000_R2.fastq.gz | samtools view -O BAM - | samtools sort -T temp -O bam -o md5638_2.sorted.bam -

Now index the BAM file:

In [ ]:
samtools index md5638_2.sorted.bam

**Note: When the symbol `-` is used above, Unix will automatically replace `-` with the output produced by the preceeding command (i.e. the command before the `|` symbol).**

## Mark PCR Duplicates

We will use a program called ‘MarkDuplicates’ that is part of Picard tools (http://picard.sourceforge.net) to remove PCR duplicates that may have been introduced during the library construction stage. To find the options for ‘MarkDuplicates’ – type:

In [ ]:
picard MarkDuplicates

Now run MarkDuplicates using the ‘I=’ option to specify the input BAM file and the ‘O=’ option to specify the output file (e.g. md5638.markdup.bam). You will also need to specify the duplication metrics output file using ‘M=’ (e.g. md5638.markdup.metrics).

In [ ]:
picard MarkDuplicates I=md5638.sorted.bam O=md5638.markdup.bam M=md5638.metrics.txt

__Q3: From looking at the output metrics file - how many reads were marked as duplicates? What
was the percent duplication?__

Don't forget to generate an index for the new bam file using samtools.

In [ ]:
samtools index md5638.markdup.bam

## Generate QC Stats

Use samtools to collect some statistics and generate QC plots from the alignment in the BAM file from the previous step. Make sure you save the output of the stats command to a file (e.g. `md5638.markdup.stats`).

In [ ]:
samtools stats md5638.markdup.bam > md5638.markdup.stats

In [ ]:
plot-bamstats -p md5638_plot/ md5638.markdup.stats

## Exercises

Now look at the output and answer the following questions:

__Q4: What is the total number of reads?__

__Q5: What proportion of the reads were mapped?__

__Q6: How many reads were paired correctly/properly?__

__Q7: How many read pairs mapped to a different chromosome?__

__Q8: What is the insert size mean and standard deviation?__

In your web browser open the file called md5638_plot.html to view the QC information and anser the following questions:

__Q9: How many reads have zero mapping quality?__

__Q10: Which of the first fragments or second fragments are higher base quality on average?__

Congratulations you have succesfully aligned some NGS data to a reference genome! Now continue to the next section of the tutorial: [Alignment visualisation](visualisation.ipynb).